In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score

In [2]:
train = pd.read_csv('/kaggle/input/walmart-sales-forecast/train.csv')
test = pd.read_csv('/kaggle/input/walmart-sales-forecast/test.csv')
features = pd.read_csv('/kaggle/input/walmart-sales-forecast/features.csv')
stores = pd.read_csv('/kaggle/input/walmart-sales-forecast/stores.csv')

In [3]:
train.head()


,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [4]:
test.head()


,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False


In [5]:
features.head()


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [6]:
features.drop(['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'],axis =1 , inplace = True )

In [7]:
stores.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [8]:
merged_df = features.merge(stores)


In [9]:
merged_df.head()

,Store,Date,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday,Type,Size
0,1,2010-02-05,42.31,2.572,211.096358,8.106,False,A,151315
1,1,2010-02-12,38.51,2.548,211.242170,8.106,True,A,151315
2,1,2010-02-19,39.93,2.514,211.289143,8.106,False,A,151315
3,1,2010-02-26,46.63,2.561,211.319643,8.106,False,A,151315
4,1,2010-03-05,46.50,2.625,211.350143,8.106,False,A,151315


In [10]:
train_merged_df = train.merge(merged_df, on=['Store', 'Date', 'IsHoliday'])
test_merged_df = test.merge(merged_df, on=['Store', 'Date', 'IsHoliday'])

In [11]:
train_merged_df['Date'] = pd.to_datetime(train_merged_df['Date'])
test_merged_df['Date'] = pd.to_datetime(test_merged_df['Date'])

In [12]:
train_merged_df['day'] = train_merged_df['Date'].dt.day
train_merged_df['month'] = train_merged_df['Date'].dt.month
train_merged_df['year'] = train_merged_df['Date'].dt.year
train_merged_df['dayofweek'] = train_merged_df['Date'].dt.dayofweek


In [13]:
test_merged_df['day'] = test_merged_df['Date'].dt.day
test_merged_df['month'] = test_merged_df['Date'].dt.month
test_merged_df['year'] = test_merged_df['Date'].dt.year
test_merged_df['dayofweek'] = test_merged_df['Date'].dt.dayofweek

In [14]:
train_merged_df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Type,Size,day,month,year,dayofweek
0,1,1,2010-02-05,24924.50,False,42.31,2.572,211.096358,8.106,A,151315,5,2,2010,4
1,1,1,2010-02-12,46039.49,True,38.51,2.548,211.242170,8.106,A,151315,12,2,2010,4
2,1,1,2010-02-19,41595.55,False,39.93,2.514,211.289143,8.106,A,151315,19,2,2010,4
3,1,1,2010-02-26,19403.54,False,46.63,2.561,211.319643,8.106,A,151315,26,2,2010,4
4,1,1,2010-03-05,21827.90,False,46.50,2.625,211.350143,8.106,A,151315,5,3,2010,4


In [15]:
train_merged_df = train_merged_df.sort_values(['Store', 'Dept', 'Date'])
for lag in [1, 7, 30]:
    train_merged_df[f'lag_{lag}'] = train_merged_df.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(lag)


In [16]:
train_merged_df['rolling_mean_7'] = train_merged_df.groupby(['Store', 'Dept'])['Weekly_Sales'].transform(lambda x: x.rolling(7, 1).mean())
train_merged_df['rolling_std_7'] = train_merged_df.groupby(['Store', 'Dept'])['Weekly_Sales'].transform(lambda x: x.rolling(7, 1).std())

In [17]:
le = LabelEncoder()
train_merged_df['Type_encoded'] = le.fit_transform(train_merged_df['Type'])
test_merged_df['Type_encoded'] = le.transform(test_merged_df['Type'])

In [18]:
train_merged_df = train_merged_df.dropna()


In [19]:
features = ['Store', 'Dept', 'day', 'month', 'year', 'dayofweek', 'Temperature', 'Fuel_Price', 'CPI', 
            'Unemployment', 'Size', 'Type_encoded']

In [20]:
X_train = train_merged_df[features]
y_train = train_merged_df['Weekly_Sales']
X_test = test_merged_df[features]

In [21]:
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

In [22]:
model = RandomForestRegressor(n_estimators=100, random_state=42, verbose=2, n_jobs=-1)
model.fit(X_train_imputed, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 100building tree 2 of 100
building tree 3 of 100

building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   27.5s


building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100building tree 70 of 100

building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.3min finished


RandomForestRegressor(n_jobs=-1, random_state=42, verbose=2)

In [23]:
predictions = model.predict(X_test_imputed)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.7s finished


In [24]:
print("Training completed successfully")
print(f"Predictions shape: {predictions.shape}")
print("Sample predictions:", predictions[:5])

Training completed successfully
Predictions shape: (115064,)
Sample predictions: [32170.3042 20232.6705 24979.8827 24390.7847 30774.6688]


In [25]:
train_predictions = model.predict(X_train_imputed)
train_r2 = r2_score(y_train, train_predictions)
print(f'Training R2 Score: {train_r2}')

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.6s


Training R2 Score: 0.9968875160263233


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.7s finished
